[모델 y=Wx+b]
1. 초기값 설정 : 가우시안 분포(정규분포)
2. 손실함수 : MSE
3. 옵티마이저 : Gradient descent (learning rate=0.01)
4. 학습 epoch = 500

In [441]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [442]:
def load_csv(file_path):

    # CSV 파일 로드 (첫 번째 행을 건너뜀, 앞에서부터 8개의 열만 불러옴)
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1, usecols=range(8))
    
    # 'HAEMOGLOBINS'를 목표 변수로 설정 (두 번째 열)
    x_data = np.delete(data, 1, axis=1)  # 두 번째 열 제거 (입력 변수)
    y_data = data[:, 1].reshape(-1, 1)  # 두 번째 열이 목표 변수
    
    return x_data, y_data

def split_data(x_data, y_data):
    # 첫 400개의 데이터를 테스트 데이터로 사용
    x_test = x_data[:400]
    y_test = y_data[:400]
    
    # 나머지 데이터를 학습 및 검증 데이터로 사용
    x_rest = x_data[400:]
    y_rest = y_data[400:]
    
    # 학습 데이터와 검증 데이터로 분할 예시 (전체의 약 12.5%를 검증 데이터로 사용)
    x_train, x_val, y_train, y_val = train_test_split(x_rest, y_rest, test_size=0.125, random_state=42)

    # 표준화 (스케일링)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_val = sc.transform(x_val)
    x_test = sc.transform(x_test)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

In [443]:
x_data, y_data = load_csv('data-ori.csv')
x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_data, y_data)

In [444]:
W = tf.Variable(tf.random_normal(shape=[x_train.shape[1], 1]), name="W")
b = tf.Variable(tf.random_normal(shape=[1]), name="b")
x = tf.placeholder(tf.float32, shape=[None, x_train.shape[1]], name="x")
y = tf.placeholder(tf.float32, shape=[None, 1], name="y")

In [445]:
# 선형회귀 모델(Wx + b)을 정의
linear_model = tf.matmul(x, W) +b

In [446]:
# 손실 함수: MSE (평균 제곱 오차)
loss = tf.reduce_mean(tf.square(linear_model - y))

In [447]:
# 최적화: 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_step = optimizer.minimize(loss)

In [448]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 500번 학습 수행
    for i in range(500):
        sess.run(train_step, feed_dict={x: x_train, y: y_train})

        if i % 50 == 0:
            current_loss = sess.run(loss, feed_dict={x: x_train, y: y_train})
            print(f"Iteration {i}, Current Loss: {current_loss}")

    # 테스트 데이터에 대한 예측 수행
    y_pred = sess.run(linear_model, feed_dict={x: x_test})

    # 차이 합 계산
    difference_sum = np.sum(np.abs(y_pred - y_test))
    print(f'차이 합: {difference_sum}')

Iteration 0, Current Loss: 164.35874938964844
Iteration 50, Current Loss: 21.069740295410156
Iteration 100, Current Loss: 2.8401577472686768
Iteration 150, Current Loss: 0.42068588733673096
Iteration 200, Current Loss: 0.09643075615167618
Iteration 250, Current Loss: 0.052241917699575424
Iteration 300, Current Loss: 0.0458984300494194
Iteration 350, Current Loss: 0.04478941485285759
Iteration 400, Current Loss: 0.044442299753427505
Iteration 450, Current Loss: 0.04421931877732277
차이 합: 15.488026237487794
